<a href="https://www.kaggle.com/code/gauthamnair2005/creating-the-dataset-top?scriptVersionId=257909479" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Best 250 Movie List Scraper from IMDb

## Loading 250 Titles

The 250 titles are loaded first by first code

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
driver = webdriver.Chrome()
driver.get("https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc")
wait = WebDriverWait(driver, 10)

## Clicking all the loading button at the bottom of the page
load_button_xpath = '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button'
clicked = 0
while True:
    try:
        button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, load_button_xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", button)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", button)
        print(f"Button clicked successfully. Total: {clicked+1}")
        clicked += 1
        time.sleep(1)
    except Exception:
        print("No more 'Show 50 more' buttons found or all titles loaded.")
        break
print(f"All 'Show 50 more' buttons clicked. Total: {clicked}")

## going to top of the page
driver.execute_script("window.scrollTo(0, 0);")

i_button_xpath='//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li[{}]/div/div/div/div[1]/div[3]/button'
try:
    for i in range(1, 251):
        button = wait.until(EC.presence_of_element_located((By.XPATH, i_button_xpath.format(i))))
        driver.execute_script("arguments[0].scrollIntoView(true);", button)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", button)
        time.sleep(1)
        popup = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[4]/div[2]/div"))
        )
        popup_html = popup.get_attribute("outerHTML")
        with open(f"scrap/popup{i}.html", "w", encoding="utf-8") as f:
            f.write(popup_html)
        time.sleep(1)
        close_button_xpath='/html/body/div[4]/div[2]/div/div[1]/button'
        close_button = wait.until(EC.presence_of_element_located((By.XPATH, close_button_xpath)))
        time.sleep(1)
        driver.execute_script("arguments[0].click();", close_button)
        time.sleep(1)
    print("All popups saved for 250 titles.")
except Exception as e:
    print(f"Error while clicking button: {e}")

driver.quit()

## The rest info fetching from popup

This is handled by second code, which fetches the information of loaded titles.

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import re

movie_name=[]
movie_year=[]
movie_time=[]
movie_type=[]
movie_rating=[]
movie_description=[]
movie_director=[]
movie_stars=[]

def clean_text(tag):
    raw_text = tag.get_text(separator=' ', strip=True)
    return re.sub(r'\s+', ' ', raw_text)

def safe_append(target_list, value):
    target_list.append(value if value else "NA")

for i in range(1,251):
    print(i)
    try:
        with open(f"scrap/popup{i}.html","r",encoding="utf-8") as f:
            content=f.read()
        soup=BeautifulSoup(content,"lxml")
        name_tags = soup.find_all("h3", class_="ipc-title__text prompt-title-text ipc-title__text--reduced")
        for tag in name_tags:
            name= clean_text(tag)
        g_data=soup.find_all("ul",class_="ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--no-wrap ipc-inline-list--inline baseAlt")
        for i in range(len(g_data)):
            if i==0:
                data=[j.get_text(strip=True) for j in g_data[i] if j.get_text(strip=True)!=""]
                year=data[0]
                time=data[1]
            else:
                type=[j.get_text(strip=True) for j in g_data[i] if j.get_text(strip=True)!=""]
                type=" ".join(type)
        rating=soup.find_all("span",class_="ipc-rating-star--rating")[0].get_text(strip=True)
        description=soup.find_all("div",class_="sc-f3a43855-2 dCytFm")[0].get_text(strip=True)
        d_data=soup.find_all("ul",class_="ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline baseAlt")
        for i in range(len(d_data)):
            if i==0:
                data=[j.get_text(separator=' ',strip=True) for j in d_data[i] if j.get_text(strip=True)!=""]
                director=re.sub(r'\s+', ' ', data[0])
            else:
                stars=[j.get_text(separator=' ',strip=True) for j in d_data[i] if j.get_text(strip=True)!=""]
                stars=" ".join(stars)
                stars=re.sub(r'\s+', ' ', stars)
    except Exception as e:
        pass
    
    safe_append(movie_name, name)
    safe_append(movie_year, year)
    safe_append(movie_time, time)
    safe_append(movie_type, type)
    safe_append(movie_rating, rating)
    safe_append(movie_description, description)
    safe_append(movie_director, director)
    safe_append(movie_stars, stars)


movie_dict = {
    "name": movie_name,
    "year": movie_year,
    "duration": movie_time,
    "genre": movie_type,
    "rating": movie_rating,
    "description": movie_description,
    "director": movie_director,
    "stars": movie_stars
}

df=pd.DataFrame(data=movie_dict)
print(df.shape)
df.to_csv("movie_data.csv",index=False)
